<a href="https://colab.research.google.com/github/ankit-rathi/Tradevesting_v1/blob/main/myProspects_Value_Buy_Sell_Backtesting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
!pip install yfinance
!pip install pandas-ta

In [38]:
import yfinance as yf
import pandas as pd
import numpy as np
import pandas_ta as ta

def find_buy_sell_levels(stock_list):
    results = []

    for stock in stock_list:
        # Download historical data from yfinance
        data = yf.download(stock + '.NS', period='10y', interval='1d', progress=False)

        if data.empty:
            print(f"No data found for {stock}")
            continue

        # Calculate 200-day SMA and 14-day RSI
        data['Close'] = round(data['Close'],2)
        data['SMA_200'] = round(data['Close'].rolling(window = 200, min_periods = 1).mean(),0)
        data.ta.rsi(append=True)
        data['RSI_14'] = round(data['RSI_14'],0)

        # All-time high (ATH)
        data['ATH'] = data['Close'].cummax()

        buy_triggered = False
        buy_date = None
        for i in range(len(data)):
            if not buy_triggered:
                # Buy criteria: Price < 200 SMA and RSI < 30 for the first time
                if data['Close'].iloc[i] < data['SMA_200'].iloc[i] and data['RSI_14'].iloc[i] < 30:
                    buy_triggered = True
                    buy_date = data.index[i]
                    results.append({
                        'Stock': stock,
                        'Date': buy_date,
                        'Close': data['Close'].iloc[i],
                        '200 SMA': data['SMA_200'].iloc[i],
                        'RSI 14': data['RSI_14'].iloc[i],
                        'Action': 'Buy'
                    })
            else:
                # Sell criteria: Price reaches ATH after buying
                if data['Close'].iloc[i] >= data['ATH'].iloc[i]:
                    sell_date = data.index[i]
                    results.append({
                        'Stock': stock,
                        'Date': sell_date,
                        'Close': data['Close'].iloc[i],
                        '200 SMA': data['SMA_200'].iloc[i],
                        'RSI 14': data['RSI_14'].iloc[i],
                        'Action': 'Sell'
                    })
                    buy_triggered = False  # Reset for next buy/sell cycle

    # Return the results as a DataFrame
    return pd.DataFrame(results)

def get_combined_signals(result):
    # Filter buy and sell actions
    buy_signals = result[result['Action'] == 'Buy'].copy()
    sell_signals = result[result['Action'] == 'Sell'].copy()

    # Sort both buy and sell signals by stock and date
    buy_signals = buy_signals.sort_values(by=['Stock', 'Date']).reset_index(drop=True)
    sell_signals = sell_signals.sort_values(by=['Stock', 'Date']).reset_index(drop=True)

    # Create an empty DataFrame to hold the combined signals
    combined_signals = []

    # Iterate over each stock to ensure correct pairing of Buy and Sell signals
    stocks = buy_signals['Stock'].unique()

    for stock in stocks:
        # Get buy and sell signals for the stock
        buy_stock = buy_signals[buy_signals['Stock'] == stock].reset_index(drop=True)
        sell_stock = sell_signals[sell_signals['Stock'] == stock].reset_index(drop=True)

        # Pair each buy with the next available sell after the buy date
        i, j = 0, 0
        while i < len(buy_stock):
            if j < len(sell_stock) and sell_stock.loc[j, 'Date'] > buy_stock.loc[i, 'Date']:
                # Valid pair of buy and sell
                combined_signals.append({
                    'Stock': stock,
                    'Buy Date': buy_stock.loc[i, 'Date'],
                    'Buy Price': buy_stock.loc[i, 'Close'],
                    'Sell Date': sell_stock.loc[j, 'Date'],
                    'Sell Price': sell_stock.loc[j, 'Close'],
                    'Gain %': round(((sell_stock.loc[j, 'Close'] - buy_stock.loc[i, 'Close']) / buy_stock.loc[i, 'Close']) * 100,2),
                    'Days Held': (sell_stock.loc[j, 'Date'] - buy_stock.loc[i, 'Date']).days
                })
                # Move to the next buy and sell pair
                i += 1
                j += 1
            else:
                # If no valid sell is found, append buy action and keep sell-related fields as None
                combined_signals.append({
                    'Stock': stock,
                    'Buy Date': buy_stock.loc[i, 'Date'],
                    'Buy Price': buy_stock.loc[i, 'Close'],
                    'Sell Date': None,
                    'Sell Price': None,
                    'Gain %': None,
                    'Days Held': None
                })
                # Move to the next buy action
                i += 1

    # Convert the combined signals list to a DataFrame
    combined_signals_df = pd.DataFrame(combined_signals)

    # Return the final result
    return combined_signals_df

import yfinance as yf
import pandas as pd
import numpy as np
import pandas_ta as ta
from scipy.optimize import newton
import datetime


# Helper function to calculate XIRR
def xirr(cashflows, dates):
    def xnpv(rate):
        return sum([cf / (1 + rate) ** ((date - dates[0]).days / 365.25) for cf, date in zip(cashflows, dates)])

    return newton(xnpv, 0.1)  # Start with an initial guess of 10%

# Function to calculate XIRR and CAGR for each row and overall
def calculate_xirr(combined_signals, initial_investment=100000):
    # List to store results for each row
    xirr_results = []

    # Add today's date and current stock price for unsold trades
    today = pd.Timestamp.today()

    for index, row in combined_signals.iterrows():
        if pd.notna(row['Sell Date']):
            # Cash flows: Buy is a negative outflow, Sell is a positive inflow
            cashflows = [-initial_investment, initial_investment * (1 + row['Gain %'] / 100)]
            dates = [row['Buy Date'], row['Sell Date']]

            # Calculate XIRR for the row
            xirr_value = round(xirr(cashflows, dates) * 100, 2)  # XIRR in percentage
            xirr_results.append(xirr_value)
        else:
            # Fetch current stock price if sell date is None
            stock_data = yf.download(row['Stock'] + '.NS', period='1d', interval='1d', progress=False)
            current_price = stock_data['Close'].iloc[0] if not stock_data.empty else row['Buy Price']

            # Cash flows with today's price as sell price
            cashflows = [-initial_investment, initial_investment * (current_price / row['Buy Price'])]
            dates = [row['Buy Date'], today]

            # Calculate XIRR for unsold stock
            xirr_value = round(xirr(cashflows, dates) * 100, 2)
            xirr_results.append(xirr_value)

            # Update the Sell Date and Sell Price in the DataFrame for unsold stock
            combined_signals.at[index, 'Sell Date'] = today
            combined_signals.at[index, 'Sell Price'] = current_price
            combined_signals.at[index, 'Gain %'] = round(((current_price - row['Buy Price']) / row['Buy Price']) * 100, 2)
            combined_signals.at[index, 'Days Held'] = (today - row['Buy Date']).days

    # Add XIRR results to the DataFrame
    combined_signals['XIRR %'] = xirr_results

    return combined_signals

def calculate_overall_xirr(combined_result):
    # List to hold cash flows (buy as positive, sell as negative) and dates
    cashflows = []
    dates = []

    # Iterate over each trade in the combined_result table
    for i, row in combined_result.iterrows():
        # For each trade, assume 1L rupees as investment
        investment = 100000

        # 1. Add Buy event
        cashflows.append(investment)  # Buy is positive cashflow (investment)
        dates.append(row['Buy Date'])

        # 2. Add Sell event if Sell Date is not None
        if pd.notna(row['Sell Date']):
            gain_percentage = row['Gain %'] / 100
            booked_value = investment * (1 + gain_percentage)  # Booked value = investment * (1 + Gain %)
            cashflows.append(-booked_value)  # Sell is negative cashflow (returning money)
            dates.append(row['Sell Date'])
        else:
            # If no sell, assume current date and use the current price as the sell price
            current_value = investment * (1 + row['Gain %'] / 100)  # Calculate current value assuming no sell
            cashflows.append(-current_value)
            dates.append(pd.Timestamp.now())  # Use today's date for open positions

    # Convert cashflows and dates to lists and pass to xirr
    overall_xirr = round(xirr(cashflows, dates) * 100, 2)

    return overall_xirr


# Example usage
if __name__ == "__main__":
    mypf = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')

    mypf = mypf[(mypf['Criteria'] == 'X40') | (mypf['Criteria'] == 'X40N')]

    stock_n100 = mypf['Symbol'].unique()

    exclude = ['CADILAHC','MMTC','JIOFIN','SCHAEFFLER ','ADANITRANS', 'EQUITAS', 'MCDOWELL-N', 'ARE&M', 'UJJIVAN']

    stock_n100 = mypf[~mypf['Symbol'].isin(exclude) ]['Symbol'].unique()

    stock_ids = stock_n100
    result = find_buy_sell_levels(stock_ids)
    combined_result = get_combined_signals(result)

    # Calculate XIRR
    combined_result = calculate_xirr(combined_result)

    # Calculate overall CAGR
    overall_xirr = calculate_overall_xirr(combined_result)

    print("Overall CAGR:", overall_xirr)


ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['ANGELONE.NS']: YFInvalidPeriodError("%ticker%: Period '10y' is invalid, must be one of ['1d', '5d', '1mo', '3mo', '6mo', '1y', '2y', '5y', 'ytd', 'max']")


No data found for ANGELONE


ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['EQUITASBNK.NS']: YFInvalidPeriodError("%ticker%: Period '10y' is invalid, must be one of ['1d', '5d', '1mo', '3mo', '6mo', '1y', '2y', '5y', 'ytd', 'max']")


No data found for EQUITASBNK


ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['INDIGOPNTS.NS']: YFInvalidPeriodError("%ticker%: Period '10y' is invalid, must be one of ['1d', '5d', '1mo', '3mo', '6mo', '1y', '2y', '5y', 'ytd', 'max']")


No data found for INDIGOPNTS


ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['UJJIVANSFB.NS']: YFInvalidPeriodError("%ticker%: Period '10y' is invalid, must be one of ['1d', '5d', '1mo', '3mo', '6mo', '1y', '2y', '5y', 'ytd', 'max']")


No data found for UJJIVANSFB
Overall CAGR: 23.4


In [39]:
combined_result

,Stock,Buy Date,Buy Price,Sell Date,Sell Price,Gain %,Days Held,XIRR %
0,3MINDIA,2016-01-25,9643.40,2016-04-12 00:00:00.000000,13228.950000,37.18,78.0,339.43
1,3MINDIA,2016-11-16,12207.80,2017-11-03 00:00:00.000000,15411.150000,26.24,352.0,27.35
2,3MINDIA,2018-10-23,19377.30,2021-03-04 00:00:00.000000,26190.900000,35.16,863.0,13.60
3,3MINDIA,2022-02-11,23061.80,2023-08-24 00:00:00.000000,30548.900000,32.47,559.0,20.17
4,5PAISA,2017-12-06,125.57,2017-12-26 00:00:00.000000,236.450000,88.30,20.0,10458028.39
...,...,...,...,...,...,...,...,...
302,VIPIND,2022-05-20,553.90,2024-09-12 06:17:13.190443,487.399994,-12.01,846.0,-5.37
303,WHIRLPOOL,2015-09-01,666.65,2016-06-21 00:00:00.000000,809.800000,21.47,294.0,27.33
304,WHIRLPOOL,2018-09-19,1495.35,2019-09-20 00:00:00.000000,1807.900000,20.90,366.0,20.85
305,WHIRLPOOL,2020-03-19,1856.15,2020-12-28 00:00:00.000000,2580.600000,39.03,284.0,52.77


In [37]:
def get_financial_year(date):
    """Returns the financial year for a given date in the format 'YYYY-YYYY'."""
    if date.month < 4:
        return f"{date.year-1}-{date.year}"
    else:
        return f"{date.year}-{date.year+1}"

# Helper function to calculate XIRR
def xirr(cashflows, dates):
    def xnpv(rate):
        return sum([cf / (1 + rate) ** ((date - dates[0]).days / 365.25) for cf, date in zip(cashflows, dates)])

    # Try different initial guesses to improve convergence
    try:
        return newton(xnpv, 0.1)  # Start with an initial guess of 10%
    except RuntimeError:
        try:
            return newton(xnpv, 0.01)  # Try a smaller initial guess
        except RuntimeError:
            return np.nan # Return NaN if it still doesn't converge


def calculate_fy_xirr(combined_result):
    # List to hold cash flows (buy as positive, sell as negative) and dates
    cashflows = []
    dates = []
    fy_mapping = []

    # Iterate over each trade in the combined_result table
    for i, row in combined_result.iterrows():
        # Assume 1L rupees as investment for each buy event
        investment = 100000

        # 1. Add Buy event
        cashflows.append(investment)
        buy_date = row['Buy Date']
        dates.append(buy_date)
        fy_mapping.append(get_financial_year(buy_date))

        # 2. Add Sell event if Sell Date is not None
        if pd.notna(row['Sell Date']):
            gain_percentage = row['Gain %'] / 100
            booked_value = investment * (1 + gain_percentage)
            cashflows.append(-booked_value)
            sell_date = row['Sell Date']
            dates.append(sell_date)
            fy_mapping.append(get_financial_year(sell_date))
        else:
            # If no sell, assume current date and calculate current value
            current_value = investment * (1 + row['Gain %'] / 100)
            cashflows.append(-current_value)
            current_date = pd.Timestamp.now()
            dates.append(current_date)
            fy_mapping.append(get_financial_year(current_date))

    # Create a DataFrame to handle the financial year grouping
    cashflow_df = pd.DataFrame({
        'Date': dates,
        'Cashflow': cashflows,
        'Financial Year': fy_mapping
    })

    # Group by financial year
    financial_years = cashflow_df['Financial Year'].unique()

    # Dictionary to hold XIRR results per financial year
    fy_xirr = {}

    # Calculate XIRR for each financial year
    for fy in financial_years:
        group = cashflow_df[cashflow_df['Financial Year'] == fy]
        # Ensure there are both positive and negative cashflows (XIRR needs at least one inflow and one outflow)
        if (group['Cashflow'] > 0).any() and (group['Cashflow'] < 0).any():

            try:
                # Check for invalid data points and calculate XIRR
                xirry = round(xirr( group['Cashflow'], group['Date']) *100,2)
                fy_xirr[fy] = xirry
            except (ValueError, OverflowError) as e:
                # Handle any errors during XIRR calculation
                fy_xirr[fy] = np.nan
        else:
            # If no valid cashflow pattern for XIRR (all positive or all negative), set XIRR as NaN
            fy_xirr[fy] = np.nan

    return fy_xirr





# Example Usage:
# Assuming `combined_result` is your dataframe with 'Buy Date', 'Sell Date', and 'Gain %'
fy_xirr_result = calculate_fy_xirr(combined_result)

# Print financial-year-wise XIRR
for fy, xirr_value in fy_xirr_result.items():
    print(f"Financial Year {fy}: XIRR = {xirr_value:.2%}")

<ipython-input-37-8ec41c941c2c>:11: RuntimeWarning: invalid value encountered in scalar power
  return sum([cf / (1 + rate) ** ((date - dates[0]).days / 365.25) for cf, date in zip(cashflows, dates)])


KeyError: 0